In [9]:
%%bash

set -e

TARGET_DIRECTORY=duquantum/
if [[ -d $TARGET_DIRECTORY ]]; then
    echo "Deleting $TARGET_DIRECTORY..."
    rm -rf $TARGET_DIRECTORY
    echo "-----------------------------------"
fi

REPO="https://github.com/Dpbm/duquantum"
echo "Cloning repo from $REPO ..."
git clone $REPO $TARGET_DIRECTORY
echo "Directory ($TARGET_DIRECTORY) data:"
ls -la $TARGET_DIRECTORY

echo "-----------------------------------"
PYTHON_VERSION=$(cat $TARGET_DIRECTORY/.python-version)
echo "You must install python version: $PYTHON_VERSION"
python_dependencies=("python$PYTHON_VERSION" "python$PYTHON_VERSION-venv")
echo "Required dependencies: ${python_dependencies[@]}"
echo "Installing apt dependencies..." && \
    sudo apt update && \
    sudo apt install -y "${python_dependencies[@]}"

Deleting duquantum/...
-----------------------------------
Cloning repo from https://github.com/Dpbm/duquantum ...
Directory (duquantum/) data:
total 68
drwxr-xr-x 4 root root  4096 Feb  1 11:59 .
drwxr-xr-x 1 root root  4096 Feb  1 11:59 ..
-rw-r--r-- 1 root root  1065 Feb  1 11:59 CMakeLists.txt
-rw-r--r-- 1 root root    63 Feb  1 11:59 conanfile.txt
-rw-r--r-- 1 root root   251 Feb  1 11:59 dev-requirements.txt
drwxr-xr-x 8 root root  4096 Feb  1 11:59 .git
-rw-r--r-- 1 root root  6150 Feb  1 11:59 .gitignore
-rw-r--r-- 1 root root 10280 Feb  1 11:59 LICENSE
-rw-r--r-- 1 root root  1553 Feb  1 11:59 Makefile
-rw-r--r-- 1 root root     5 Feb  1 11:59 .python-version
-rw-r--r-- 1 root root     0 Feb  1 11:59 README.md
-rw-r--r-- 1 root root  1047 Feb  1 11:59 statevec.cu
-rw-r--r-- 1 root root  6223 Feb  1 11:59 statevec.hpp
drwxr-xr-x 2 root root  4096 Feb  1 11:59 tests
-----------------------------------
You must install python version: 3.13
Required dependencies: python3.13 python

Cloning into 'duquantum'...


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)




In [2]:
! nvidia-smi

Sun Feb  1 11:39:27 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
%%bash

set -e

CUDA_VERSION=$(nvcc --version | tail -n 1 | grep -Eo 'cuda_[0-9]{2}' | sed 's|cuda_||g')
echo "Using cuda ${CUDA_VERSION}!"

cd duquantum
echo "DuQuantum directory ($(pwd)):"
ls -la

python3.13 -m venv .venv
source .venv/bin/activate

echo "------------------------------"
echo "Installing dependencies..."
pip install -r dev-requirements.txt
pip install "cuquantum-cu${CUDA_VERSION}"


Using cuda 12!
DuQuantum directory (/content/duquantum):
total 68
drwxr-xr-x 4 root root  4096 Feb  1 11:59 .
drwxr-xr-x 1 root root  4096 Feb  1 11:59 ..
-rw-r--r-- 1 root root  1065 Feb  1 11:59 CMakeLists.txt
-rw-r--r-- 1 root root    63 Feb  1 11:59 conanfile.txt
-rw-r--r-- 1 root root   251 Feb  1 11:59 dev-requirements.txt
drwxr-xr-x 8 root root  4096 Feb  1 11:59 .git
-rw-r--r-- 1 root root  6150 Feb  1 11:59 .gitignore
-rw-r--r-- 1 root root 10280 Feb  1 11:59 LICENSE
-rw-r--r-- 1 root root  1553 Feb  1 11:59 Makefile
-rw-r--r-- 1 root root     5 Feb  1 11:59 .python-version
-rw-r--r-- 1 root root     0 Feb  1 11:59 README.md
-rw-r--r-- 1 root root  1047 Feb  1 11:59 statevec.cu
-rw-r--r-- 1 root root  6223 Feb  1 11:59 statevec.hpp
drwxr-xr-x 2 root root  4096 Feb  1 11:59 tests
------------------------------
Installing dependencies...
  Using cached certifi-2026.1.4-py3-none-any.whl.metadata (2.5 kB)
  Using cached charset_normalizer-3.4.4-cp313-cp313-manylinux2014_x86_64.man

In [11]:
%%bash

set -e

echo "Installing dev dependencies..."
sudo apt update && sudo apt install -y build-essential cmake make binutils clang ninja-build
echo "-----------------------------------------------"

echo "building project and running tests..."
cd duquantum
echo "DuQuantum directory ($(pwd)):"
ls -la
echo "-----------------------------------------------"

source .venv/bin/activate

make setup-conan

PYTHON_VERSION=$(cat .python-version)
CUQUANTUM_PATH="$(pwd)/.venv/lib/python${PYTHON_VERSION}/site-packages/cuquantum"

CUQUANTUM_INCLUDE_DIRS="$CUQUANTUM_PATH/include"
CUQUANTUM_LIB_DIRS="$CUQUANTUM_PATH/lib"
CUDA_INCLUDE_DIRS="/usr/local/cuda/include"

echo "-----------------------------------------------"
echo "USING: "
echo "PYTHON VERSION: $PYTHON_VERSION"
echo "CUQUANTUM PATH: $CUQUANTUM_PATH"
echo "CUQUANTUM INCLUDE DIRS: $CUQUANTUM_INCLUDE_DIRS"
echo "CUQUANTUM LIB DIRS: $CUQUANTUM_LIB_DIRS"
echo "CUDA INCLUDE DIRS: $CUDA_INCLUDE_DIRS"
echo "-----------------------------------------------"

echo "CUQUANTUM_LIB_DIRS files:"
ls -la $CUQUANTUM_LIB_DIRS
echo "-----------------------------------------------"

modules=("statevec")
for module in ${modules[@]}; do
    TARGET_VERSION="libcu${module}.so"

    if [[ ! -f "$CUQUANTUM_LIB_DIRS/${TARGET_VERSION}" ]]; then
        FIRST_VERSION=$(ls $CUQUANTUM_LIB_DIRS | grep $module | head -n 1)
        FIRST_VERSION_PATH="$CUQUANTUM_LIB_DIRS/$FIRST_VERSION"
        TARGET_VERSION_PATH="$CUQUANTUM_LIB_DIRS/$TARGET_VERSION"
        echo "Fixing ${FIRST_VERSION}(${FIRST_VERSION_PATH}) linking to ${TARGET_VERSION_PATH}..."
        ln -s $FIRST_VERSION_PATH $TARGET_VERSION_PATH
    fi
done
echo "-----------------------------------------------"

make delete && \
    make CUDA_INCLUDE_DIRS=$CUDA_INCLUDE_DIRS CUQUANTUM_INCLUDE_DIRS=$CUQUANTUM_INCLUDE_DIRS CUQUANTUM_LIB_DIRS=$CUQUANTUM_LIB_DIRS
echo "-----------------------------------------------"

echo "BUILD dir data:"
ls -la ./build

echo "BUILD tests dir data:"
ls -la ./build/tests

make run-tests



Installing dev dependencies...
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (3,198 B/s)
Reading package lists...
Building dependency tree...
Reading state information...
102 packages can be upgraded. Run 'apt list --upgradable' to see them.
Readi



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


detect_api: Found cc=gcc-11.4.0
detect_api: gcc>=5, using the major as version
detect_api: gcc C++ standard library: libstdc++11

Detected profile:
WARN: This profile is a guess of your environment, please check it.
WARN: The output of this command is not guaranteed to be stable and can change in future Conan versions.
WARN: Use your own profile files for stability.
Saving detected profile to /root/.conan2/profiles/default

======== Input profiles ========
Profile host:
[settings]
arch=x86_64
build_type=Debug
compiler=gcc
compiler.cppstd=gnu17
compiler.libcxx=libstdc++11
compiler.version=11
os=Linux

Profile build:
[settings]
arch=x86_64
build_type=Release
compiler=gcc
compiler.cppstd=gnu17
compiler.libcxx=libstdc++11
compiler.version=11
os=Linux


======== Computing dependency graph ========
Gra

CalledProcessError: Command 'b'\nset -e\n\necho "Installing dev dependencies..."\nsudo apt update && sudo apt install -y build-essential cmake make binutils clang ninja-build\necho "-----------------------------------------------"\n\necho "building project and running tests..."\ncd duquantum\necho "DuQuantum directory ($(pwd)):"\nls -la\necho "-----------------------------------------------"\n\nsource .venv/bin/activate\n\nmake setup-conan\n\nPYTHON_VERSION=$(cat .python-version)\nCUQUANTUM_PATH="$(pwd)/.venv/lib/python${PYTHON_VERSION}/site-packages/cuquantum"\n\nCUQUANTUM_INCLUDE_DIRS="$CUQUANTUM_PATH/include"\nCUQUANTUM_LIB_DIRS="$CUQUANTUM_PATH/lib"\nCUDA_INCLUDE_DIRS="/usr/local/cuda/include"\n\necho "-----------------------------------------------"\necho "USING: "\necho "PYTHON VERSION: $PYTHON_VERSION"\necho "CUQUANTUM PATH: $CUQUANTUM_PATH"\necho "CUQUANTUM INCLUDE DIRS: $CUQUANTUM_INCLUDE_DIRS"\necho "CUQUANTUM LIB DIRS: $CUQUANTUM_LIB_DIRS"\necho "CUDA INCLUDE DIRS: $CUDA_INC